## Basic Setup

In [14]:
#Dependicies
import time
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from config import path

In [15]:
#setup chromedriver
executable_path = {"executable_path":f"{path}"}
browser=Browser("chrome", **executable_path, headless=True)

## Scraping Mars News Site

In [16]:
#Mars news URL to be scraped
mars_news_page='https://mars.nasa.gov/news/'
browser.visit(mars_news_page)
#wait a moment for the page to be redirected
time.sleep(1)

html=browser.html
news_soup=bs(html,'html.parser')
#print(html)

In [17]:
#Retrieve Title and teaser paragraph

news_title=news_soup.find_all('div',class_='content_title')[1].text
news_p=news_soup.find_all('div',class_="article_teaser_body")[0].text

print(news_title)
print('-----------------------------')
print(news_p)

NASA's Perseverance Rover Is Midway to Mars 
-----------------------------
Sometimes half measures can be a good thing – especially on a journey this long. The agency's latest rover only has about 146 million miles left to reach its destination.


## Scrap JPL Featured Mars Space Images

In [5]:
#JPL image URL to be scraped
jpl_url='https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)
html=browser.html
image_soup=bs(html,'html.parser')

In [6]:
#Retrieve Image URL
relative_img_path=image_soup.find_all('img')[3]['src']
featured_image=f"https://www.jpl.nasa.gov{relative_img_path}"
print(featured_image)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA24189-640x350.jpg


## Scrap Mars Facts

In [7]:
facts_url='https://space-facts.com/mars/'
tables = pd.read_html(facts_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [8]:
mars_facts_df=tables[0]
mars_facts_df.columns =['Description','Value']
mars_facts_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [9]:
mars_fact_html=mars_facts_df.to_html()
mars_fact_html.replace('\n','')
print(mars_fact_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Description</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
    

## Scraping Mars Hemisphere Images

In [10]:
hemi_url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemi_url)
html=browser.html
hemi_soup=bs(html,'html.parser')

In [11]:
hemi_url=[]

items = hemi_soup.find_all('div', class_='item')

for i in items:
    hemisphere = i.find('div',class_='description')
    title = hemisphere.h3.text
    
    hemisphere_link=hemisphere.a['href']
    browser.visit(f'https://astrogeology.usgs.gov{hemisphere_link}')
    
    time.sleep(1)
    
    image_html=browser.html
    image_soup=bs(image_html,'html.parser')
    
    image_url=image_soup.find('div',class_='downloads').find('li').a['href']
    
    hemi_url.append({'title':title,'img_url':image_url})
    
print(hemi_url)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


## Putting it all together

In [12]:
mars_dict={
    "news_title":news_title,
    "news_p":news_p,
    "featured_image":featured_image,
    "fact_table":str(mars_fact_html),
    "hemisphere_images":hemi_url
}
mars_dict

{'news_title': "NASA's Perseverance Rover Is Midway to Mars ",
 'news_p': "Sometimes half measures can be a good thing – especially on a journey this long. The agency's latest rover only has about 146 million miles left to reach its destination.",
 'featured_image': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA24189-640x350.jpg',
 'fact_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Di

In [13]:
browser.quit()